# --- Day 10: Factory ---


In [ ]:
# --- Support Functions ---
# This section contains support functions used by the main code.

import re
import itertools

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

class Machine:
    def __init__(self, line: str):
        self.target, self.buttons, self.joltage = self._parse_machine(line)
        self.nlights = len(self.target)      # number of light of the indicator
        self.nbuttons = len(self.buttons)   # number of buttons
        self.button_vectors = self._make_vectors()
        self.best = None
        self.best_combo = None

    def _parse_machine(self, line):
        line = line.strip()
        # Lights Indicator pattern
        lights_string = re.search(r"\[(.*?)\]", line).group(1)
        target = [1 if c == '#' else 0 for c in lights_string]

        # Buttons
        button_strings = re.findall(r"\((.*?)\)", line)
        buttons = []
        for b in button_strings:
            b = b.strip()
            buttons.append([int(x) for x in b.split(",")])

        # Joltage
        joltage_string = re.search(r"\{(.*?)\}", line).group(1)
        joltage = [int(x) for x in joltage_string.split(",")]
                
        return target, buttons, joltage

    def _make_vectors(self):
        """Convert button toggle definition in boolean vectors"""
        vectors = []
        for button in self.buttons:
            vec = [0] * self.nlights
            for i in button:
                vec[i] = 1
            vectors.append(vec)
        return vectors

    def min_presses(self):
        """Solve via brute-force over 2^m subsets (fine for ~20 buttons)."""
        best = None

        for mask in range(1 << self.nbuttons):
            presses = mask.bit_count()

            # prune if already worse
            if best is not None and presses >= best:
                continue

            # compute results of pressing buttons in mask
            result = [0] * self.nlights
            for i in range(self.nbuttons):
                if (mask >> i) & 1:
                    for j in range(self.nlights):
                        result[j] ^= self.button_vectors[i][j]

            if result == self.target:
                best = presses

        return best

    def min_presses_joltage(self):
        """
        Try all combinations of button presses up to max_presses total.
        """

        k = len(self.joltage)
        m = self.nbuttons

        best = None

        max_total = sum(self.joltage)
        min_total = max(self.joltage)
        
        print("Presses:", end=" ")
        # Try all possible press distributions
        for total in range(min_total,max_total + 1):
            print(total,end=" ")

            if best is not None and total >= best:
                break

            for combo in itertools.product(range(total + 1), repeat=m):
                # combo is a combination of all possible num of pressing for each button
                # ex a combo [1,3,4,1] means button1 pressed 1 time, button 2 pressed 3 times...
                #if sum(combo) != total:
                #    continue

                if sum(combo) != total:
                    continue

                if best is not None and totla >= best:
                    break

                result = [0] * k
                overflow = False

                for i in range(m):
                    ci = combo[i]
                    if ci == 0:
                        continue

                    for j in range(k):
                        result[j] += self.button_vectors[i][j] * ci

                        # If exceeds target → overflow → skip outer loop correctly
                        if result[j] > self.joltage[j]:
                            overflow = True
                            break
                    
                    if overflow:
                        break
                
                if overflow:
                    continue

                if result == self.joltage:
                    best = total
                    print("\nSolution fund with best ", best, "presses:", combo)
                    break

        return best   # not found within bounds

    def min_presses_joltage2(self):
        self.best = None
        self.best_combo = None
        self._dfs(0, [0] * len(joltage), 0, [])

    def _dfs(self, i, current, presses, combo):
        # bounding : higher than best
        if self.best is not None and presses >= self.best:
            return

        # bounding: a component is higher than target
        for c, t in zip(current, self.joltage):
            if c > t:
                return

        # all buttons are used
        if i == len(self.buttons):
            if current == self.joltage:
                self.best = presses
                self.best_combo = combo.copy()
            return

        # calculate max presses for button i
        max_press = max(
            (self.joltage[j] - current[j]) // self.button_vectors[i][j] if self.button_vectors[i][j] > 0 else 0
            for j in range(len(self.joltage))
        )

        # test all possible pressings from 0 to max_press
        for p in range(max_press + 1):
            new_current = [current[j] + self.button_vectors[i][j]*p for j in range(len(self.joltage))]
            combo.append(p)
            self._dfs(i+1, new_current, presses+p, combo)
            combo.pop()


In [ ]:
# --- Part ONE ---

input = read_input('input.txt')

result = 0

machines = []
for l in input:
    machine = Machine(l)
    result += machine.min_presses()


print("Part ONE:", result)

In [ ]:
# --- Part TWO ---

# Questa soluzione con brute force non è utilizzabile per troppe iterazioni
input = read_input('input.txt')

result = 0

i = 1
for l in input:

    machine = Machine(l)
    print (f"Search solution for machine {i}/{len(input)}")
    machine.min_presses_joltage2()
    if machine.best is None:
        raise ValueError("Solution not found within limits.")
    result += machine.best
    i +=1

print("Part TWO:", result)

In [ ]:
%pip install ortools

# --- Part TWO ---

from ortools.linear_solver import pywraplp

# -----------------------------
# Data
# -----------------------------
button_vectors = [
    (3,6),
    (0,1,2,3,4,5,7,9),
    (0,1,5,6,7,8,9),
    (1,9),
    (0,1,3,4,5,6,7),
    (0,1,2,3,4,5),
    (1,2,3,4,5,6,7,8),
    (2,3,5,7,8),
    (2,3,5,7,9),
    (0,1,2,3,4,6,9),
    (4,5,6,7,8),
    (3,6,7,8,9)
]

joltage = [52,67,66,109,49,65,70,66,33,72]
dim = len(joltage)
nbuttons = len(button_vectors)

# convert button vectors into 0/1 matrix
matrix = []
for t in button_vectors:
    v = [0]*dim
    for i in t:
        v[i] = 1
    matrix.append(v)

# -----------------------------
# ILP solver
# -----------------------------
solver = pywraplp.Solver.CreateSolver("CBC")
press = [solver.IntVar(0, solver.infinity(), f"p{i}") for i in range(nbuttons)]

# constraints
for j in range(dim):
    solver.Add(sum(press[i] * matrix[i][j] for i in range(nbuttons)) == joltage[j])

# objective
solver.Minimize(solver.Sum(press))

status = solver.Solve()

# -----------------------------
# Result
# -----------------------------
if status == pywraplp.Solver.OPTIMAL:
    print("Minimum presses:", solver.Objective().Value())
    print("Press vector:", [int(p.solution_value()) for p in press])
else:
    print("No solution found")

In [ ]:
import itertools

max_values = [6, 3, 2, 5, 7]

# Genera intervalli (range) con limite superiore incluso
ranges = [range(0, m+1) for m in max_values]

combinations = list(itertools.product(*ranges))

print(len(combinations))   # numero totale
print(combinations[:10])   # primi 10 (per controllo)